Docker Command: docker run -e "ACCEPT_EULA=Y" -e "MSSQL_SA_PASSWORD=SQLServer123@" -p 1433:1433 --name sqlserver -d mcr.microsoft.com/mssql/server:2022-latest
Connection String:
Server = localhost,1433; Database = RMS-Project; User Id = sa; Password = SQLServer123@; TrustServerCertificate=True

Bảng user

In [ ]:
CREATE TABLE Users
(
    UserID UNIQUEIDENTIFIER PRIMARY KEY DEFAULT NEWID(),
    Username NVARCHAR(50) NOT NULL,
    PasswordHash NVARCHAR(128) NOT NULL,
    PasswordSalt NVARCHAR(128) NOT NULL,
    RoleID INT FOREIGN KEY REFERENCES Roles(RoleID)
)

Bảng Role

In [ ]:
CREATE TABLE Roles
(
    RoleID INT PRIMARY KEY ,
    RoleName NVARCHAR(50) NOT NULL
)

In [ ]:
DROP TABLE Users;
DROP TABLE Roles;

Dữ liệu mẫu bảng Roles

In [14]:
INSERT INTO Roles(RoleID, RoleName) VALUES (1, N'COMPANY');
INSERT INTO Roles(RoleID, RoleName) VALUES (2, N'CANDIDATE');
INSERT INTO Roles(RoleID, RoleName) VALUES (3, N'EMPLOYEE');

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.015

Stored Procedues User

In [ ]:
CREATE PROCEDURE RegisterUser
    @Username NVARCHAR(50),
    @PasswordHash NVARCHAR(128),
    @PasswordSalt NVARCHAR(128),
    @RoleID INT
AS
BEGIN
    INSERT INTO Users(Username, PasswordHash, PasswordSalt, RoleID)
    VALUES (@Username, @PasswordHash, @PasswordSalt, @RoleID)
END

In [ ]:
CREATE PROCEDURE LoginUser
    @Username NVARCHAR(50),
    @PasswordHash NVARCHAR(128)
AS
BEGIN
    SELECT UserID, RoleID FROM Users
    WHERE Username = @Username AND PasswordHash = @PasswordHash
END